In [4]:
from skimage.feature import hog
from skimage.transform import pyramid_gaussian
import joblib
from skimage import color
from imutils.object_detection import non_max_suppression
import imutils
import numpy as np
import cv2
import os
import glob
import time
import pandas as pd
import random
random.seed(42)
np.random.seed(42)

In [2]:
train_df = pd.read_csv('../dataset/processed/train.csv', sep = ',')
val_df = pd.read_csv('../dataset/processed/validation.csv', sep = ',')


In [3]:
train_df

,name,num_faces,bbox
0,0--Parade/0_Parade_Parade_0_904.jpg,1,361 98 263 339
1,0--Parade/0_Parade_marchingband_1_799.jpg,21,78 221 7 8
2,0--Parade/0_Parade_marchingband_1_799.jpg,21,78 238 14 17
3,0--Parade/0_Parade_marchingband_1_799.jpg,21,113 212 11 15
4,0--Parade/0_Parade_marchingband_1_799.jpg,21,134 260 15 15
...,...,...,...
126002,9--Press_Conference/9_Press_Conference_Press_C...,1,256 195 243 355
126003,9--Press_Conference/9_Press_Conference_Press_C...,1,222 26 578 742
126004,9--Press_Conference/9_Press_Conference_Press_C...,1,322 253 358 418
126005,9--Press_Conference/9_Press_Conference_Press_C...,1,462 140 264 362


In [69]:
startTime = time.time()
num_faces = 0
for i,row in train_df.iterrows():
    
    if i % 1000 == 0:
        print(i)
    if i >= 33000:
        file_name = '../dataset/images/' + row['name']
        img = cv2.imread(file_name)
        bbox = row['bbox']
        bbox = bbox.split(" ")
        bbox = [int(num) for num in bbox]
    #     print(bbox)
        x_min, y_min, width, height = bbox[0], bbox[1], bbox[2], bbox[3]
        face = img[y_min:y_min+height, x_min:x_min+width]
        face_path = '../dataset/hog_svm/cropped_positive/' + str(num_faces) + ".jpg"
        try:
            cv2.imwrite(face_path, face)
        except:
            pass
    num_faces += 1

    #     cv2.imshow("Resized image", face)
    #     cv2.waitKey(0)
    #     cv2.destroyAllWindows()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000


In [7]:
def sliding_window(image, stepSize, windowSize):# image is the input, step size is the no.of pixels needed to skip and windowSize is the size of the actual window
    # slide a window across the image
    for y in range(0, image.shape[0], stepSize):# this line and the line below actually defines the sliding part and loops over the x and y coordinates
        for x in range(0, image.shape[1], stepSize):
            # yield the current window
            yield (x, y, image[y: y + windowSize[1], x:x + windowSize[0]])


In [8]:
def is_overlapping_1D(line1, line2):
    """
    line:
        (xmin, xmax)
    """
    return line1[0] <= line2[1] and line2[0] <= line1[1]

def is_overlapping_2d(box1, box2):
    """
    box:
        (xmin, ymin, xmax, ymax)
    """
    return is_overlapping_1D([box1[0],box1[2]],[box2[0],box2[2]]) and is_overlapping_1D([box1[1],box1[3]],[box2[1],box2[3]])


In [44]:
def check_in_bbox(x,y, windowW, windowH, bboxes):
    in_bbox = False
    x_min, y_min = x,y
    x_max, y_max = x_min + windowW, y_min + windowH
    for bbox in bboxes:
        x_box_min, y_box_min, box_width, box_height = bbox[0], bbox[1], bbox[2], bbox[3]
        x_box_max, y_box_max = x_box_min + box_width, y_box_min + box_height
        if is_overlapping_2d( (x_min, y_min, x_max, y_max), (x_box_min, y_box_min, x_box_max, y_box_max), ):
            in_bbox = True
            break
    return in_bbox


In [18]:
def random_position_in_image(image_shape, window_shape):
    return (
        randint(0, image_shape[0] - window_shape[0]), 
        randint(0, image_shape[1] - window_shape[1])
    )


In [19]:
from random import randint
for i in range(1000):
    print(random_position_in_image((512,512), (36,36)))

(197, 258)
(230, 391)
(281, 143)
(422, 319)
(348, 312)
(60, 65)
(49, 201)
(191, 406)
(173, 285)
(187, 386)
(73, 101)
(308, 260)
(205, 256)
(20, 23)
(19, 70)
(365, 170)
(411, 242)
(265, 233)
(76, 310)
(457, 263)
(71, 167)
(476, 313)
(163, 83)
(201, 315)
(378, 431)
(153, 303)
(172, 259)
(424, 260)
(272, 250)
(362, 288)
(153, 243)
(417, 8)
(188, 169)
(344, 56)
(213, 298)
(157, 407)
(460, 371)
(446, 352)
(322, 13)
(305, 242)
(135, 335)
(400, 397)
(296, 295)
(116, 369)
(26, 298)
(245, 87)
(268, 322)
(369, 317)
(396, 430)
(194, 75)
(420, 348)
(124, 16)
(293, 358)
(56, 97)
(9, 225)
(160, 214)
(77, 211)
(353, 104)
(210, 256)
(396, 313)
(471, 241)
(446, 434)
(376, 372)
(31, 361)
(70, 265)
(106, 287)
(166, 339)
(244, 269)
(192, 160)
(88, 235)
(467, 272)
(175, 279)
(181, 346)
(395, 444)
(369, 348)
(329, 411)
(354, 135)
(312, 247)
(98, 126)
(142, 285)
(152, 115)
(152, 394)
(147, 360)
(106, 353)
(360, 250)
(162, 245)
(178, 286)
(407, 369)
(140, 147)
(62, 293)
(346, 278)
(194, 458)
(202, 419)
(176, 

In [62]:
36 * 1.5

54.0

In [66]:
num_windows_chosen = 0

for i in range(len(train_df['name'].unique())):
    img_name = train_df['name'].unique()[i]
    if num_windows_chosen % 2000 == 0:
        print(num_windows_chosen)
    file_name = '../dataset/images/' + img_name
    img = cv2.imread(file_name)
    width, height = img.shape[1], img.shape[0]
    if width < 288 or height < 288:
        continue
#     img = cv2.resize(img, (512,512))
    bbox_column = train_df[train_df['name'] == img_name]['bbox']
    bboxes = []
    for box in bbox_column:
        box = box.split(' ')
        box = [int(num) for num in box]
        bboxes.append(box)
#         print(box)
#         cv2.rectangle(img,(box[0], box[1]), (box[0] + box[2], box[1] + box[3]), (255, 0, 0), 2)
#     cv2.imshow("Resized image", img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

    num_faces = list(train_df[train_df['name'] == img_name]['num_faces'])[0]
    
    non_faces_windows = []
    
    small_windows = 0
    window_shape = (72,72)
    for i in range(10000):
        if small_windows > 10:
            break
        rand_x, rand_y = random_position_in_image((width,height), window_shape)
        patch = img[rand_y: rand_y + window_shape[1], rand_x:rand_x + window_shape[0]]
        if check_in_bbox(rand_x,rand_y,window_shape[1],window_shape[0], bboxes) == False:
            non_faces_windows.append(patch)
#             cv2.imshow("Resized image", patch)
#             cv2.waitKey(0)
#             cv2.destroyAllWindows()

            small_windows += 1
    
    window_shape = (144,144)
    medium_windows = 0
    for i in range(500):
        if medium_windows > 10:
            break
        rand_x, rand_y = random_position_in_image((width,height), window_shape)
        patch = img[rand_y: rand_y + window_shape[1], rand_x:rand_x + window_shape[0]]
        if check_in_bbox(rand_x,rand_y,window_shape[1],window_shape[0], bboxes) == False:
            non_faces_windows.append(patch)
            medium_windows += 1

            
    window_shape = (288,288)
    big_windows = 0
    for i in range(200):
        if big_windows > 5:
            break
        rand_x, rand_y = random_position_in_image((width,height), window_shape)
        patch = img[rand_y: rand_y + window_shape[1], rand_x:rand_x + window_shape[0]]
        if check_in_bbox(rand_x,rand_y,window_shape[1],window_shape[0], bboxes) == False:
            non_faces_windows.append(patch)
            big_windows += 1

#     window_shape = (288,288)
#     big_windows = 0
#     for i in range(200):
#         if big_windows > 5:
#             break
#         rand_x, rand_y = random_position_in_image((width,height), window_shape)
#         patch = img[rand_y: rand_y + window_shape[1], rand_x:rand_x + window_shape[0]]
#         if check_in_bbox(rand_x,rand_y,window_shape[1],window_shape[0], bboxes) == False:
#             non_faces_windows.append(patch)
#             big_windows += 1

    
#     print(len(non_faces_windows))
#             small_windows += 1
#         cv2.imshow("Resized image", patch)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
#     break
    
#     for (x,y,window) in sliding_window(img, stepSize=5, windowSize=(72,72)):
#         # if the window does not meet our desired window size, ignore it!
#         if small_windows >= 2:
#             break
#         if window.shape[0] != 72 or window.shape[1] != 72 or window.shape[2] != 3: # ensure the sliding window has met the minimum size requirement
#             continue
#         if check_in_bbox(x,y,72,72, bboxes) == False:
#             non_faces_windows.append(window)
#             small_windows += 1
    
#     medium_windows = 0
#     for (x,y,window) in sliding_window(img, stepSize=5, windowSize=(128,128)):
#         # if the window does not meet our desired window size, ignore it!
#         if medium_windows >= 2:
#             break
#         if window.shape[0] != 128 or window.shape[1] != 128 or window.shape[2] != 3: # ensure the sliding window has met the minimum size requirement
#             continue
#         if check_in_bbox(x,y,128,128, bboxes) == False:
#             non_faces_windows.append(window)
#             medium_windows += 1

#     big_windows = 0
#     for (x,y,window) in sliding_window(img, stepSize=5, windowSize=(256,256)):
#         if big_windows >= 2:
#             break

#         # if the window does not meet our desired window size, ignore it!
#         if window.shape[0] != 256 or window.shape[1] != 256 or window.shape[2] != 256: # ensure the sliding window has met the minimum size requirement
#             continue
#         if check_in_bbox(x,y,256,256, bboxes) == False:
#             non_faces_windows.append(window)
#             big_windows += 1

    
    max_windows_chosen = 10
    if len(non_faces_windows) > max_windows_chosen:
        chosen_windows = random.sample(non_faces_windows, max_windows_chosen)
    else:
        chosen_windows = non_faces_windows
#     print(len(chosen_windows))
    for window in chosen_windows:
        window_path = '../dataset/hog_svm/cropped_negative_alt/' + str(num_windows_chosen) + ".jpg"
        try:
#             pass
            cv2.imwrite(window_path, window)
        except:
            pass
        num_windows_chosen += 1
        
    if num_windows_chosen >= 100000:
        break
#     print(max_windows_chosen)
#     print(len(non_faces_windows))    
#             print(0)
#             cv2.imshow("Resized image", window)
#             cv2.waitKey(0)
#             cv2.destroyAllWindows()

#         cv2.imshow("Window", window)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()

            



0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000


In [2]:
import dlib
detector = dlib.get_frontal_face_detector()

def convert_and_trim_bb(image, rect):
	# extract the starting and ending (x, y)-coordinates of the
	# bounding box
	startX = rect.left()
	startY = rect.top()
	endX = rect.right()
	endY = rect.bottom()
	# ensure the bounding box coordinates fall within the spatial
	# dimensions of the image
	startX = max(0, startX)
	startY = max(0, startY)
	endX = min(endX, image.shape[1])
	endY = min(endY, image.shape[0])
	# compute the width and height of the bounding box
	w = endX - startX
	h = endY - startY
	# return our bounding box coordinates
	return (startX, startY, w, h)


In [9]:
# load the input image from disk, resize it, and convert it from
# BGR to RGB channel ordering (which is what dlib expects)
# image = cv2.imread()
image= cv2.imread("../dataset/images/21--Festival/21_Festival_Festival_21_719.jpg")
# image= cv2.imread("../dataset/images/8--Election_Campain/8_Election_Campain_Election_Campaign_8_38.jpg")
# im= cv2.imread("../dataset/images/44--Aerobics/44_Aerobics_Aerobics_44_259.jpg")
# image = cv2.imread("../dataset/images/45--Balloonist/45_Balloonist_Balloonist_45_20.jpg")
# image = cv2.imread("../dataset/images/3--Riot/3_Riot_Riot_3_308.jpg")
# image = cv2.imread("../dataset/images/3--Riot/3_Riot_Riot_3_898.jpg")
# image = cv2.imread("../dataset/images/42--Car_Racing/42_Car_Racing_Nascar_42_624.jpg")
# image = cv2.imread("../dataset/images/49--Greeting/49_Greeting_peoplegreeting_49_187.jpg")

image = imutils.resize(image, width=600)
rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# perform face detection using dlib's face detector
start = time.time()
print("[INFO[ performing face detection with dlib...")
rects = detector(rgb, 1)
end = time.time()
print("[INFO] face detection took {:.4f} seconds".format(end - start))


[INFO[ performing face detection with dlib...
[INFO] face detection took 0.4234 seconds


In [10]:
boxes = [convert_and_trim_bb(image, r) for r in rects]
# loop over the bounding boxes
for (x, y, w, h) in boxes:
	# draw the bounding box on our image
	cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
# show the output image
cv2.imshow("Output", image)
cv2.waitKey(0)

-1